<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/GPT2_Chatbot_Pruning_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Sep  3 17:08:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install tqdm
!pip install kobert-transformers==0.4.1
!pip install kogpt2-transformers==0.3.0
!pip install transformers==3.0.2
!pip install torch
!pip install tokenizers==0.8.1rc1
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.5 MB/s 
     |████████████████████████████████| 120 kB 60.8 MB/s 
     |████████████████████████████████| 6.6 MB 48.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 8.1 MB/s 
     |████████████████████████████████| 880 kB 61.8 MB/s 
     |████████████████████████████████| 3.0 MB 66.3 MB/s 
     |████████████████████████████████| 1.3 MB 84.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=da02c5807e36fb91a38a9b35e489b27e3349dbcc83b68f6b971d48332d6b3404
  Stored in directory: /root/.cache/pip/

In [3]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 84 kB/s 
     |████████████████████████████████| 453 kB 93.3 MB/s 


In [4]:
# 구글 드라이브 연결
import os
import numpy as np
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/HelloNewWorld/gpt"

Mounted at /content/gdrive/


In [5]:
import os
import numpy as np
import torch
import torch.nn as nn
from kogpt2_transformers import get_kogpt2_tokenizer, get_kogpt2_model
import torch.nn.utils.prune as prune

In [6]:
class DialogKoGPT2(nn.Module):
  def __init__(self):
    super(DialogKoGPT2, self).__init__()
    self.kogpt2 = get_kogpt2_model()

  def generate(self,
               input_ids,
               do_sample=True,
               max_length= 60,
               top_p=0.92,
               top_k=50,
               temperature= 0.6,
               no_repeat_ngram_size = None,
               num_return_sequences=2,
               early_stopping=False,
               ):
    return self.kogpt2.generate(input_ids,
               do_sample=do_sample,
               max_length=max_length,
               top_p = top_p,
               top_k=top_k,
               temperature=temperature,
               no_repeat_ngram_size= no_repeat_ngram_size,
               num_return_sequences=num_return_sequences,
               early_stopping = early_stopping,
              )

  def forward(self, input, labels = None):
    if labels is not None:
      outputs = self.kogpt2(input, labels=labels)
    else:
      outputs = self.kogpt2(input)

    return outputs

In [7]:
# Global Pruning 적용
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

model = DialogKoGPT2()
model.to(device)

parameters_to_prune = ()
for i in range(12):
    parameters_to_prune += (
        (model.kogpt2.transformer.h[i].attn.c_attn, 'weight'),
        (model.kogpt2.transformer.h[i].attn.c_proj, 'weight'),
    )

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [8]:
# Global Pruning을 통해 전체 Sparsity를 20%로 만들 수 있음을 알 수 있음.

for i in range(12):
    print(
        "Sparsity in Layer {}-th c_attn weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(model.kogpt2.transformer.h[i].attn.c_attn.weight == 0))
            / float(model.kogpt2.transformer.h[i].attn.c_attn.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th c_proj weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(model.kogpt2.transformer.h[i].attn.c_proj.weight == 0))
            / float(model.kogpt2.transformer.h[i].attn.c_proj.weight.nelement())
        )
    )
    print()

    
numerator, denominator = 0, 0
for i in range(12):
    numerator += torch.sum(model.kogpt2.transformer.h[i].attn.c_attn.weight == 0)
    numerator += torch.sum(model.kogpt2.transformer.h[i].attn.c_proj.weight == 0)

    denominator += model.kogpt2.transformer.h[i].attn.c_attn.weight.nelement()
    denominator += model.kogpt2.transformer.h[i].attn.c_proj.weight.nelement()
    
print("Global sparsity: {:.2f}%".format(100. * float(numerator) / float(denominator)))

Sparsity in Layer 1-th c_attn weight: 28.47%
Sparsity in Layer 1-th c_proj weight: 65.74%

Sparsity in Layer 2-th c_attn weight: 20.61%
Sparsity in Layer 2-th c_proj weight: 24.94%

Sparsity in Layer 3-th c_attn weight: 18.73%
Sparsity in Layer 3-th c_proj weight: 27.17%

Sparsity in Layer 4-th c_attn weight: 17.68%
Sparsity in Layer 4-th c_proj weight: 23.86%

Sparsity in Layer 5-th c_attn weight: 17.33%
Sparsity in Layer 5-th c_proj weight: 21.64%

Sparsity in Layer 6-th c_attn weight: 17.04%
Sparsity in Layer 6-th c_proj weight: 19.34%

Sparsity in Layer 7-th c_attn weight: 17.24%
Sparsity in Layer 7-th c_proj weight: 18.92%

Sparsity in Layer 8-th c_attn weight: 17.60%
Sparsity in Layer 8-th c_proj weight: 19.29%

Sparsity in Layer 9-th c_attn weight: 16.98%
Sparsity in Layer 9-th c_proj weight: 19.14%

Sparsity in Layer 10-th c_attn weight: 17.09%
Sparsity in Layer 10-th c_proj weight: 17.16%

Sparsity in Layer 11-th c_attn weight: 17.65%
Sparsity in Layer 11-th c_proj weight: 15.

In [9]:
root_path=path
data_path = f"{root_path}/data/chatbot_wellness_dialog_for_autoregressive.txt"
checkpoint_path =f"{root_path}/checkpoint"
# save_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive-20210922-add-chatbotdata.pth"
save_ckpt_path = f"{checkpoint_path}/kogpt2-20220710-add-chatbotdata.pth"

ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_ckpt_path, map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

tokenizer = get_kogpt2_tokenizer()

count = 0
output_size = 200 # 출력하고자 하는 토큰 갯수

Downloading:   0%|          | 0.00/920k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/849k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

In [22]:
while 1:
#for i in range(5):
  sent = input('Question: ')  # '요즘 기분이 우울한 느낌이에요'
  tokenized_indexs = tokenizer.encode(sent)

  input_ids = torch.tensor([tokenizer.bos_token_id,]  + tokenized_indexs +[tokenizer.eos_token_id]).unsqueeze(0).to(device)
  # set top_k to 50
  sample_output = model.generate(
      input_ids=input_ids
      ).to(device)


  print("Answer: " + tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True))
  print(100 * '-')
  # print(sample_output.shape)

# for s in kss.split_sentences(sent):
#     print(s)

Question: 안 그래도 몸이 안 좋은데 잠까지 설치니까 진짜…너무 힘들어.


Answer: 많이 힘들어 보여요. 힘든 일이 있으면 저에게 털어놓으세요.많이 지쳤나봐요. 곧 기운낼 수 있을 거예요.많이 지쳤나봐요. 곧 기운낼
----------------------------------------------------------------------------------------------------
Question: 앉아 있으면 그나마 좀 덜해서 침대에 거의 앉아서 자는데 그것도 너무 힘들어요.


Answer: 가슴이 답답하겠어요. 얼른 시간이 지나버렸으면 좋겠어요.가슴이 답답하면 정말 생활이 불편하죠. 스트레스를 너무 많이 받는다면 고민이 많으시겠어요.
----------------------------------------------------------------------------------------------------
Question: 너무 힘들었어요.


Answer: 많이 힘들어 보여요. 힘든 일이 있으면 저에게 털어놓으세요.많이 힘들어 보여요. 힘든 일이 있으면 저에게 털어놓으세요.많이 답답하시겠어요.그런 일이 있으셨군요.그런 일이 있으셨
----------------------------------------------------------------------------------------------------
Question: 그런데 그래도 힘들고.. 너무 지쳐.


Answer: 그런 일이 있으셨군요. 조금씩 나아질 거라고 믿어요.힘든 일은 오래 겪으셨어요. 조금씩 나아질 거라고 믿어요.지금은 괜찮으세요? 아직 많이 힘드시겠어요. 하지만 저는
----------------------------------------------------------------------------------------------------
Question: 최근에 아픈 날이 많아진 거 같아.


...

Answer: 그렇군요. 서로 오래 모른다고나 할까요?좀 더 알아보고 하세요.더 들려주세요. 더 말씀해주세요. 더 말씀해주세요. 더 말씀해주세요. 더 말씀해주세요.
----------------------------------------------------------------------------------------------------
Question: 그 새끼가 옆에 젊은 여자 끼고 술을 마시고 있더라구.


Answer: 그렇군요. 고민이 많으시겠어요. 다른 방법으로 노력을 해보는 건 어떠세요? 용기를 내봐요. 지금 만나보세요. 네, 그렇군요. 그러면요. 조금 기다
----------------------------------------------------------------------------------------------------
Question: 약을 끊었더니 증상이 예전보다 더 심해진 것 같아.


Answer: 너무 심하시면 병원을 다시 가보는 건 어떨까요? 검사 결과는 좋으면 계속 곁에 있어요. 당신의 마음도 조금 나아질 거예요. 당신의 마음도 조금 나아질 거예요. 당신의 마음도 변화가 있으
----------------------------------------------------------------------------------------------------
Question: 특별히 뭐 잘 못 먹은 것도 아닌데 속이 안 좋아요.


Answer: 소화 기능에 뭔가 문제가 생긴 건 아닐까요? 병원 진찰을 권유해드립니다. 당신의 건망증이 심하셨군요. 혹시 괜찮으신가요? 혹시 괜찮으시다면 저에게 말해보세요
----------------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored

In [37]:
from konlpy.tag import Mecab
from nltk.translate.bleu_score import sentence_bleu

mecab = Mecab()

result = []

In [38]:
# 안 그래도 몸이 안 좋은데 잠까지 설치니까 진짜…너무 힘들어.	많이 힘들어 보여요. 힘든 일이 있으면 저에게 털어놓으세요.
# 안 그래도 몸이 안 좋은데 잠까지 설치니까 진짜…너무 힘들어.	그런 일이 있으셨군요. 조금씩 나아질 거라고 믿어요.
# 안 그래도 몸이 안 좋은데 잠까지 설치니까 진짜…너무 힘들어.	가슴이 답답하겠어요. 얼른 시간이 지나버렸으면 좋겠어요.
# 안 그래도 몸이 안 좋은데 잠까지 설치니까 진짜…너무 힘들어.	잘 버티고 있어요. 조금 힘들더라도 조금만 기운을 내요.

reference = ['많이 힘들어 보여요.', '힘든 일이 있으면 저에게 털어놓으세요.', '그런 일이 있으셨군요.', '조금씩 나아질 거라고 믿어요.', '가슴이 답답하겠어요.', '얼른 시간이 지나버렸으면 좋겠어요.', '잘 버티고 있어요.', '조금 힘들더라도 조금만 기운을 내요.']
reference = [mecab.morphs(s) for s in reference]

candidate = mecab.morphs('많이 힘들어 보여요. 힘든 일이 있으면 저에게 털어놓으세요.많이 지쳤나봐요. 곧 기운낼 수 있을 거예요.많이 지쳤나봐요.')

print(f'reference: {reference}', )
print('candidate:', candidate)
print('BLEU:', sentence_bleu(reference, candidate))
result.append(sentence_bleu(reference, candidate))

reference: [['많이', '힘들', '어', '보여요', '.'], ['힘든', '일', '이', '있', '으면', '저', '에게', '털어놓', '으세요', '.'], ['그런', '일', '이', '있', '으셨', '군요', '.'], ['조금', '씩', '나아질', '거', '라고', '믿', '어요', '.'], ['가슴', '이', '답답', '하', '겠', '어요', '.'], ['얼른', '시간', '이', '지나', '버렸으면', '좋', '겠', '어요', '.'], ['잘', '버티', '고', '있', '어요', '.'], ['조금', '힘들', '더라도', '조금', '만', '기운', '을', '내', '요', '.']]
candidate: ['많이', '힘들', '어', '보여요', '.', '힘든', '일', '이', '있', '으면', '저', '에게', '털어놓', '으세요', '.', '많이', '지쳤', '나', '봐요', '.', '곧', '기운', '낼', '수', '있', '을', '거', '예요', '.', '많이', '지쳤', '나', '봐요', '.']
BLEU: 0.3744387052073659


In [39]:
# 앉아 있으면 그나마 좀 덜해서 침대에 거의 앉아서 자는데 그것도 너무 힘들어요.	많이 힘들어 보여요. 힘든 일이 있으면 저에게 털어놓으세요.
# 앉아 있으면 그나마 좀 덜해서 침대에 거의 앉아서 자는데 그것도 너무 힘들어요.	그런 일이 있으셨군요. 조금씩 나아질 거라고 믿어요.
# 앉아 있으면 그나마 좀 덜해서 침대에 거의 앉아서 자는데 그것도 너무 힘들어요.	가슴이 답답하겠어요. 얼른 시간이 지나버렸으면 좋겠어요.
# 앉아 있으면 그나마 좀 덜해서 침대에 거의 앉아서 자는데 그것도 너무 힘들어요.	잘 버티고 있어요. 조금 힘들더라도 조금만 기운을 내요.

reference = ['많이 힘들어 보여요.', '힘든 일이 있으면 저에게 털어놓으세요.', '그런 일이 있으셨군요.', '조금씩 나아질 거라고 믿어요.', '가슴이 답답하겠어요.', '얼른 시간이 지나버렸으면 좋겠어요.', '잘 버티고 있어요.', '조금 힘들더라도 조금만 기운을 내요.']
reference = [mecab.morphs(s) for s in reference]

candidate = mecab.morphs('가슴이 답답하겠어요. 얼른 시간이 지나버렸으면 좋겠어요.가슴이 답답하면 정말 생활이 불편하죠. 스트레스를 너무 많이 받는다면 고민이 많으시겠어요.')

print(f'reference: {reference}', )
print('candidate:', candidate)
print('BLEU:', sentence_bleu(reference, candidate))
result.append(sentence_bleu(reference, candidate))

reference: [['많이', '힘들', '어', '보여요', '.'], ['힘든', '일', '이', '있', '으면', '저', '에게', '털어놓', '으세요', '.'], ['그런', '일', '이', '있', '으셨', '군요', '.'], ['조금', '씩', '나아질', '거', '라고', '믿', '어요', '.'], ['가슴', '이', '답답', '하', '겠', '어요', '.'], ['얼른', '시간', '이', '지나', '버렸으면', '좋', '겠', '어요', '.'], ['잘', '버티', '고', '있', '어요', '.'], ['조금', '힘들', '더라도', '조금', '만', '기운', '을', '내', '요', '.']]
candidate: ['가슴', '이', '답답', '하', '겠', '어요', '.', '얼른', '시간', '이', '지나', '버렸으면', '좋', '겠', '어요', '.', '가슴', '이', '답답', '하', '면', '정말', '생활', '이', '불편', '하', '죠', '.', '스트레스', '를', '너무', '많이', '받', '는다면', '고민', '이', '많', '으시', '겠', '어요', '.']
BLEU: 0.2898720273466421


In [40]:
# 너무 힘들었어요.	많이 힘들어 보여요. 힘든 일이 있으면 저에게 털어놓으세요.
# 너무 힘들었어요.	그런 일이 있으셨군요. 조금씩 나아질 거라고 믿어요.
# 너무 힘들었어요.	가슴이 답답하겠어요. 얼른 시간이 지나버렸으면 좋겠어요.
# 너무 힘들었어요.	잘 버티고 있어요. 조금 힘들더라도 조금만 기운을 내요.

reference = ['많이 힘들어 보여요.', '힘든 일이 있으면 저에게 털어놓으세요.', '그런 일이 있으셨군요.', '조금씩 나아질 거라고 믿어요.', '가슴이 답답하겠어요.', '얼른 시간이 지나버렸으면 좋겠어요.', '잘 버티고 있어요.', '조금 힘들더라도 조금만 기운을 내요.']
reference = [mecab.morphs(s) for s in reference]

candidate = mecab.morphs('많이 힘들어 보여요. 힘든 일이 있으면 저에게 털어놓으세요.많이 답답하시겠어요.그런 일이 있으셨군요.')

print(f'reference: {reference}', )
print('candidate:', candidate)
print('BLEU:', sentence_bleu(reference, candidate))
result.append(sentence_bleu(reference, candidate))

reference: [['많이', '힘들', '어', '보여요', '.'], ['힘든', '일', '이', '있', '으면', '저', '에게', '털어놓', '으세요', '.'], ['그런', '일', '이', '있', '으셨', '군요', '.'], ['조금', '씩', '나아질', '거', '라고', '믿', '어요', '.'], ['가슴', '이', '답답', '하', '겠', '어요', '.'], ['얼른', '시간', '이', '지나', '버렸으면', '좋', '겠', '어요', '.'], ['잘', '버티', '고', '있', '어요', '.'], ['조금', '힘들', '더라도', '조금', '만', '기운', '을', '내', '요', '.']]
candidate: ['많이', '힘들', '어', '보여요', '.', '힘든', '일', '이', '있', '으면', '저', '에게', '털어놓', '으세요', '.', '많이', '답답', '하', '시', '겠', '어요', '.', '그런', '일', '이', '있', '으셨', '군요', '.']
BLEU: 0.625683743070839


In [41]:
# 그런데 그래도 힘들고.. 너무 지쳐.	많이 힘들어 보여요. 힘든 일이 있으면 저에게 털어놓으세요.
# 그런데 그래도 힘들고.. 너무 지쳐.	그런 일이 있으셨군요. 조금씩 나아질 거라고 믿어요.
# 그런데 그래도 힘들고.. 너무 지쳐.	가슴이 답답하겠어요. 얼른 시간이 지나버렸으면 좋겠어요.
# 그런데 그래도 힘들고.. 너무 지쳐.	잘 버티고 있어요. 조금 힘들더라도 조금만 기운을 내요.

reference = ['많이 힘들어 보여요.', '힘든 일이 있으면 저에게 털어놓으세요.', '그런 일이 있으셨군요.', '조금씩 나아질 거라고 믿어요.', '가슴이 답답하겠어요.', '얼른 시간이 지나버렸으면 좋겠어요.', '잘 버티고 있어요.', '조금 힘들더라도 조금만 기운을 내요.']
reference = [mecab.morphs(s) for s in reference]

candidate = mecab.morphs('그런 일이 있으셨군요. 조금씩 나아질 거라고 믿어요.힘든 일은 오래 겪으셨어요.지금은 괜찮으세요? 아직 많이 힘드시겠어요.')

print(f'reference: {reference}', )
print('candidate:', candidate)
print('BLEU:', sentence_bleu(reference, candidate))
result.append(sentence_bleu(reference, candidate))

reference: [['많이', '힘들', '어', '보여요', '.'], ['힘든', '일', '이', '있', '으면', '저', '에게', '털어놓', '으세요', '.'], ['그런', '일', '이', '있', '으셨', '군요', '.'], ['조금', '씩', '나아질', '거', '라고', '믿', '어요', '.'], ['가슴', '이', '답답', '하', '겠', '어요', '.'], ['얼른', '시간', '이', '지나', '버렸으면', '좋', '겠', '어요', '.'], ['잘', '버티', '고', '있', '어요', '.'], ['조금', '힘들', '더라도', '조금', '만', '기운', '을', '내', '요', '.']]
candidate: ['그런', '일', '이', '있', '으셨', '군요', '.', '조금', '씩', '나아질', '거', '라고', '믿', '어요', '.', '힘든', '일', '은', '오래', '겪', '으셨어요', '.', '지금', '은', '괜찮', '으세요', '?', '아직', '많이', '힘드', '시', '겠', '어요', '.']
BLEU: 0.4023191894496083


In [42]:
# 최근에 아픈 날이 많아진 거 같아.	최근에 건강 때문에 힘드시군요. 건강을 잃은 것만큼 힘든 일은 없죠… 기운 내세요.
# 최근에 아픈 날이 많아진 거 같아.	다시 건강한 삶을 찾을 수 있을 거예요. 걱정 마세요.
# 최근에 아픈 날이 많아진 거 같아.	그러셨군요. 몸이 아프면 덩달아 마음도 힘들어지는 것 같아요.
# 최근에 아픈 날이 많아진 거 같아.	그럴 때일수록 식사와 수면이 중요하다는 거 아시죠? 잘 버텨내보자구요.

reference = ['최근에 건강 때문에 힘드시군요.', '건강을 잃은 것만큼 힘든 일은 없죠 기운 내세요.', '다시 건강한 삶을 찾을 수 있을 거예요.', '걱정 마세요.', '몸이 아프면 덩달아 마음도 힘들어지는 것 같아요.', '그럴 때일수록 식사와 수면이 중요하다는 거 아시죠? 잘 버텨내보자구요.']
reference = [mecab.morphs(s) for s in reference]

candidate = mecab.morphs('그러셨군요. 몸이 아프면 덩달아 마음도 힘들어지는 것 같아요.그런 일이 있으셨군요. 조금씩 나아질 거라고 믿어요. 지금은 괜찮으신가요? 괜찮으시다니 정말 다행이에요.')

print(f'reference: {reference}', )
print('candidate:', candidate)
print('BLEU:', sentence_bleu(reference, candidate))
result.append(sentence_bleu(reference, candidate))

reference: [['최근', '에', '건강', '때문', '에', '힘드', '시', '군요', '.'], ['건강', '을', '잃', '은', '것', '만큼', '힘든', '일', '은', '없', '죠', '기운', '내', '세요', '.'], ['다시', '건강', '한', '삶', '을', '찾', '을', '수', '있', '을', '거', '예요', '.'], ['걱정', '마세요', '.'], ['몸', '이', '아프', '면', '덩달', '아', '마음', '도', '힘들', '어', '지', '는', '것', '같', '아요', '.'], ['그럴', '때', '일수록', '식사', '와', '수면', '이', '중요', '하', '다는', '거', '아', '시', '죠', '?', '잘', '버텨', '내', '보', '자구요', '.']]
candidate: ['그러', '셨', '군요', '.', '몸', '이', '아프', '면', '덩달', '아', '마음', '도', '힘들', '어', '지', '는', '것', '같', '아요', '.', '그런', '일', '이', '있', '으셨', '군요', '.', '조금', '씩', '나아질', '거', '라고', '믿', '어요', '.', '지금', '은', '괜찮', '으신', '가요', '?', '괜찮', '으시', '다니', '정말', '다행', '이', '에요', '.']
BLEU: 0.335027349183485


In [43]:
# 돈 버는 건 힘들어도 쓰는 건 한순간이더라구.	그럴 땐 정말 막막한 것 같아요. 기운 내세요.
# 돈 버는 건 힘들어도 쓰는 건 한순간이더라구.	마음이 힘드시겠어요.
# 돈 버는 건 힘들어도 쓰는 건 한순간이더라구.	돈이 없어도 되는 세상이 얼른 왔으면 좋겠어요.
# 돈 버는 건 힘들어도 쓰는 건 한순간이더라구.	가슴이 답답하시겠어요. 그럴 때는 든든히 먹고 머리를 비우는 것도 좋은 방법이에요.

reference = ['그럴 땐 정말 막막한 것 같아요.', '기운 내세요.', '마음이 힘드시겠어요.', '돈이 없어도 되는 세상이 얼른 왔으면 좋겠어요.', '가슴이 답답하시겠어요.', '그럴 때는 든든히 먹고 머리를 비우는 것도 좋은 방법이에요.']
reference = [mecab.morphs(s) for s in reference]

candidate = mecab.morphs('마음이 힘드시겠어요. 그럴 때는 정말 기댈 곳을 찾고 싶어져요. 주저앉지 마세요. 그곳에서 배우는게 좋겠어요. 용기를 드릴게요.')

print(f'reference: {reference}', )
print('candidate:', candidate)
print('BLEU:', sentence_bleu(reference, candidate))
result.append(sentence_bleu(reference, candidate))

reference: [['그럴', '땐', '정말', '막막', '한', '것', '같', '아요', '.'], ['기운', '내', '세요', '.'], ['마음', '이', '힘드', '시', '겠', '어요', '.'], ['돈', '이', '없', '어도', '되', '는', '세상', '이', '얼른', '왔', '으면', '좋', '겠', '어요', '.'], ['가슴', '이', '답답', '하', '시', '겠', '어요', '.'], ['그럴', '때', '는', '든든히', '먹', '고', '머리', '를', '비우', '는', '것', '도', '좋', '은', '방법', '이', '에요', '.']]
candidate: ['마음', '이', '힘드', '시', '겠', '어요', '.', '그럴', '때', '는', '정말', '기댈', '곳', '을', '찾', '고', '싶', '어', '져요', '.', '주저앉', '지', '마세요', '.', '그곳', '에서', '배우', '는', '게', '좋', '겠', '어요', '.', '용기', '를', '드릴게요', '.']
BLEU: 0.23366176703090105


In [44]:
# 남친한테 먼저 헤어지자고 했어. 	그러셨군요. 힘들겠지만 잘 이겨내야죠.
# 남친한테 먼저 헤어지자고 했어. 	많이 힘들겠어요. 제가 옆에 있어드릴게요.
# 남친한테 먼저 헤어지자고 했어. 	언젠가는 이별이 찾아오기 마련이죠… 괜찮으신가요?

reference = ['그러셨군요. 힘들겠지만 잘 이겨내야죠.', '많이 힘들겠어요.', '제가 옆에 있어드릴게요.', '언젠가는 이별이 찾아오기 마련이죠... 괜찮으신가요?']
reference = [mecab.morphs(s) for s in reference]

candidate = mecab.morphs('그렇군요. 서로 오래 모른다고나 할까요? 좀 더 알아보고 하세요.더 들려주세요. 더 말씀해주세요.')

print(f'reference: {reference}', )
print('candidate:', candidate)
print('BLEU:', sentence_bleu(reference, candidate))
result.append(sentence_bleu(reference, candidate))

reference: [['그러', '셨', '군요', '.', '힘들', '겠', '지만', '잘', '이겨', '내', '야죠', '.'], ['많이', '힘들', '겠', '어요', '.'], ['제', '가', '옆', '에', '있', '어', '드릴게요', '.'], ['언젠가', '는', '이별', '이', '찾아오', '기', '마련', '이', '죠', '.', '..', '괜찮', '으신', '가요', '?']]
candidate: ['그렇', '군요', '.', '서로', '오래', '모른다고', '나', '할까요', '?', '좀', '더', '알아보', '고', '하', '세요', '.', '더', '들려', '주', '세요', '.', '더', '말씀', '해', '주', '세요', '.']
BLEU: 4.098292351100502e-155


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [45]:
# 그 새끼가 옆에 젊은 여자 끼고 술을 마시고 있더라구.	충격이 크셨겠어요. 
# 그 새끼가 옆에 젊은 여자 끼고 술을 마시고 있더라구.	절대 용서하지 마세요. 제가 대신 욕해드릴게요.
# 그 새끼가 옆에 젊은 여자 끼고 술을 마시고 있더라구.	배신감이 클 것 같아요. 괜찮으세요?

reference = ['충격이 크셨겠어요.', '절대 용서하지 마세요.', '제가 대신 욕해드릴게요.', '배신감이 클 것 같아요.', '괜찮으세요?']
reference = [mecab.morphs(s) for s in reference]

candidate = mecab.morphs('그렇군요. 고민이 많으시겠어요. 다른 방법으로 노력을 해보는 건 어떠세요? 용기를 내봐요. 지금 만나보세요. 네, 그렇군요.')

print(f'reference: {reference}', )
print('candidate:', candidate)
print('BLEU:', sentence_bleu(reference, candidate))
result.append(sentence_bleu(reference, candidate))

reference: [['충격', '이', '크', '셨', '겠', '어요', '.'], ['절대', '용서', '하', '지', '마세요', '.'], ['제', '가', '대신', '욕해', '드릴게요', '.'], ['배신감', '이', '클', '것', '같', '아요', '.'], ['괜찮', '으세요', '?']]
candidate: ['그렇', '군요', '.', '고민', '이', '많', '으시', '겠', '어요', '.', '다른', '방법', '으로', '노력', '을', '해', '보', '는', '건', '어떠세요', '?', '용기', '를', '내', '봐요', '.', '지금', '만나', '보', '세요', '.', '네', ',', '그렇', '군요', '.']
BLEU: 1.5096412088766302e-78


In [46]:
# 약을 끊었더니 증상이 예전보다 더 심해진 것 같아.	상태가 더 안 좋아지셨군요. 걱정이 되네요.
# 약을 끊었더니 증상이 예전보다 더 심해진 것 같아.	정말 힘드시겠어요. 지금도 증상이 심하신가요?
# 약을 끊었더니 증상이 예전보다 더 심해진 것 같아.	너무 심하시면 병원을 다시 가보는 건 어떨까요?

reference = ['상태가 더 안 좋아지셨군요.', '걱정이 되네요.', '정말 힘드시겠어요.', '지금도 증상이 심하신가요?', '너무 심하시면 병원을 다시 가보는 건 어떨까요?']
reference = [mecab.morphs(s) for s in reference]

candidate = mecab.morphs('너무 심하시면 병원을 다시 가보는 건 어떨까요? 검사 결과는 좋으면 계속 곁에 있어요. 당신의 마음도 조금 나아질 거예요.')

print(f'reference: {reference}', )
print('candidate:', candidate)
print('BLEU:', sentence_bleu(reference, candidate))
result.append(sentence_bleu(reference, candidate))

reference: [['상태', '가', '더', '안', '좋', '아', '지', '셨', '군요', '.'], ['걱정', '이', '되', '네요', '.'], ['정말', '힘드', '시', '겠', '어요', '.'], ['지금', '도', '증상', '이', '심하', '신가요', '?'], ['너무', '심하', '시', '면', '병원', '을', '다시', '가', '보', '는', '건', '어떨까', '요', '?']]
candidate: ['너무', '심하', '시', '면', '병원', '을', '다시', '가', '보', '는', '건', '어떨까', '요', '?', '검사', '결과', '는', '좋', '으면', '계속', '곁', '에', '있', '어요', '.', '당신', '의', '마음', '도', '조금', '나아질', '거', '예요', '.']
BLEU: 0.41578372496714516


In [47]:
# 특별히 뭐 잘 못 먹은 것도 아닌데 속이 안 좋아요.	소화 기능에 뭔가 문제가 생긴 건 아닐까요? 병원 진찰을 권유해드립니다.
# 특별히 뭐 잘 못 먹은 것도 아닌데 속이 안 좋아요.	속이 불편하시군요. 천천히 심호흡을 하면 조금은 편해질 수도 있어요.
# 특별히 뭐 잘 못 먹은 것도 아닌데 속이 안 좋아요.	속이 좋지 않으면 정말 힘들죠. 병원에 가보는 건 어떨까요?

reference = ['소화 기능에 뭔가 문제가 생긴 건 아닐까요?', '병원 진찰을 권유해드립니다.', '속이 불편하시군요.', '천천히 심호흡을 하면 조금은 편해질 수도 있어요.', '속이 좋지 않으면 정말 힘들죠.', '병원에 가보는 건 어떨까요?']
reference = [mecab.morphs(s) for s in reference]

candidate = mecab.morphs('소화 기능에 뭔가 문제가 생긴 건 아닐까요? 병원 진찰을 권유해드립니다. 당신의 건망증이 심하셨군요. 혹시 괜찮으신가요? 혹시 괜찮으시다면 저에게 말해보세요')

print(f'reference: {reference}', )
print('candidate:', candidate)
print('BLEU:', sentence_bleu(reference, candidate))
result.append(sentence_bleu(reference, candidate))

reference: [['소화', '기능', '에', '뭔가', '문제', '가', '생긴', '건', '아닐까요', '?'], ['병원', '진찰', '을', '권유', '해', '드립니다', '.'], ['속', '이', '불편', '하', '시', '군요', '.'], ['천천히', '심호흡', '을', '하', '면', '조금', '은', '편해질', '수', '도', '있', '어요', '.'], ['속', '이', '좋', '지', '않', '으면', '정말', '힘들', '죠', '.'], ['병원', '에', '가', '보', '는', '건', '어떨까', '요', '?']]
candidate: ['소화', '기능', '에', '뭔가', '문제', '가', '생긴', '건', '아닐까요', '?', '병원', '진찰', '을', '권유', '해', '드립니다', '.', '당신', '의', '건망증', '이', '심하', '셨', '군요', '.', '혹시', '괜찮', '으신', '가요', '?', '혹시', '괜찮', '으시', '다면', '저', '에게', '말', '해', '보', '세요']
BLEU: 0.3800528767347385


In [48]:
print("BLEU 10개 평균 : ", sum(result)/10)

BLEU 10개 평균 :  0.30568393829907253
